#Luiz Carlos Ferreira Carvalho
#DRE: 120025788

# IMPORTS/CACHE

*PRECISA RODAR ANTES*

In [3]:
#@title
import numpy as np
from fractions import Fraction
from functools import lru_cache, wraps
from random import randint

#Códigos encontrados no google, referencia no final do projeto

#Código para cachear função e otimizar a performance
def cache(f):

    def g(*args):
        if args not in g.cache:
            g.cache[args] = f(*args)
        return g.cache[args]
    g.cache = {}
    g.__doc__  = f.__doc__
    g.__name__ = f.__name__
    return g

#Código para cachear função que recebe um array e otimizar a performance
def np_cache(function):
    @lru_cache()
    def cached_wrapper(hashable_array):
        array = np.array(hashable_array)
        return function(array)

    @wraps(function)
    def wrapper(array):
        return cached_wrapper(tuple(array))

    wrapper.cache_info = cached_wrapper.cache_info
    wrapper.cache_clear = cached_wrapper.cache_clear

    return wrapper


# LETRA A

**LETRA A**
Enunciado:

a) Explique como que o cenário acima descrito pode ser modelado por uma cadeia de Markov, descrevendo detalhadamente os estados de sua cadeia e exibindo a sua matriz de probabilidades de transiçãao. Cada passo de sua construçãao deve ser descrito de forma detalhada.

**LETRA A**
Resposta:

Uma cadeia de Markov é uma sequência de váriaveis aleatórias, $X_0, X_1, X_2, ..., X_n$ onde dizemos que cada uma delas representa um estado $i$ em um determinado tempo $n$, e existe uma probabilidade fixa ($P_{ij}$) de que ela esteja a seguir em outro estado $j$. Matematicamente para $i_0$,...,$i_{n-1}$,$i$, $j$, temos:

$$P( X_{n+1}=j | X_n=i,X_{n-1}=i_{n-1},..., X_1=i_1,X_0=i_0)  = P_{ij}$$ 

$$→$$ 

$$P_{ij}(n) = P(X_{n+1} = x_j | X_n = x_i) $$

Onde os valores $P_{ij}$, $0 ≤ i ≤ M, 0 ≤ j ≤ N $ são chamados de probabilidades de transição da cadeia de Markov satisfazendo a propriedade de que o seu somatório deva ser igual a 1 para cada linha da matriz,

$$P_{ij}≥0$$  

$$\sum_{j=0}^{N} P_{ij} = 1$$ 

$$i = 0,1,...,M$$

Assim temos a matriz de transição:

  $$ 
 P = \begin{bmatrix}
 P_{0,0} & P_{0,1}  & P_{0,2} & \cdots  & P_{0,N}  &  \\
 P_{1,0} & P_{1,1}  & P_{1,2} & \cdots  & P_{1,N}  &  \\
 P_{2,0} & P_{2,1}  & P_{2,2} & \cdots  & P_{2,N} \\
 \vdots  & \vdots  & \vdots   & \ddots  & \vdots  \\
 P_{M,0} & P_{M,1} & P_{M,2}  & \cdots  & P_{M,N}
\end{bmatrix}
$$

Sendo assim parece tentador e intuitivo usar uma cadeia de Markov para jogos de tabuleiros simples (Apenas um numero $N$ de casas comuns, e um par de dados honestos), onde o próximo estado do jogador (casa do tabuleiro) só depende exclusivamente do estado atual (casa do tabuleiro) e do que ele tirar nos dados, ou seja, não leva em consideração o passado, por exemplo para um jogador no estado $X_n$ pouco importa seus estados anteriores $X_{n-1}$ ou ainda $X_{n-2}$, o que importa é que seu proximo estado depende apenas do rolar dos dados, e de seu estado atual, pois com essas informações saberemos para onde ele se moverá.

Podemos então modelar um jogo de tabuleiro simples com uma cadeia de Makorv, pois a casa atual do jogador será a linha da matriz ($i$) de trnasição em que ele está, e cada coluna ($j$) representando as casas que ele poderá alcançar após sua jogada atual. Nesta modelagem $P_{ij}$ será a probabilidade de ir para cada nova casa, e este valor é obtido ao analisar os casos favoraveis e possiveis das rolagens dos dados, dessa forma teremos um valor fixo para tais probabilidades, por exemplo, um jogador na casa 4 independente do tempo que se passou sempre terá a mesma probabilidade de nesta rodada alcançar a casa 10, pois o par de dados se manterá o mesmo (Não confundir com a probabilidade de estar na casa 4 após $n$ rodadas).

O Banco Imobiliário por sua vez não é um jogo de tabuleiro simples, mesmo que em sua versão simplificada para este projeto:

Regras do jogo:

* Tabuleiro de 40 casas
* Jogador começa na casa 0 e vai até a casa 39
* Jogador rola uma dupla de dados honesto de 6 faces cada e se movimenta conforme o valor encontrado
* Quando tirar uma dupla de valores iguais (ex: 2 + 2) o jogador não se move e atira novamente o dado
* Tirando 3 duplas de valores iguais nos dados (ex: 2+2, 1+1, 3+3) o jogador estará preso tendo que ir para a casa 20 do tabuleiro, está representando a prisão
* Para sair da prisão o jogador precisa tirar uma duplas de valores iguais ("A mesma sorte que o colocou, o tirará")
* Após 3 insucessos o jogador sairá automaticamente da prisão continunando na casa 20, porém agora no estado "Visita a Prisão"
* O jogador ao sair da prisão rola novamente os dados para de fato andar
* Se o jogador cair na cassa 20 após uma rolagem de dados distintos, este não irá preso, e sim estará visitando a prisão

Assim podemos ver que o próximo estado do jogador depende do valor dos dados da rodada anterior, isto se deve a regra da prisão, já que um jogador que tira dados com valores iguais, ou seja, 2 + 2 não altera seu estado, tendo que rolar novamente os dados, e tirando mais uma dupla repetida, este poderá ser preso no próximo turno $n$ sendo mandando diretamente pra casa de numero 20, esta regra a principio impediria a modelagem da cadeia de Markov pois estamos dependendo de eventos que ocorreram no passado $X_{n-1}$ ou ainda $X_{n-2}$, para saber a próxima casa do jogador $X_{n+1}$.

Para contornar tal problema optei por uma modelagem da cadeia de Markov com uma matriz de transição 123x123, com isso podemos registrar os estados do jogador quando este tira uma dupla de dados e quando está preso, e depender apenas de seu estado $X_n$ para saber seu possivel próximo estado. Para representar os estados em que o jogador tirou uma dupla de dados, os chamaremos de estados auxiliares e usaremos um apóstrofo $'$ referenciado de "linha" para diferencia-los, na matriz de transição serão as linhas de 40 a 79, enquanto os estados auxiliares para quando o jogador tirar duas duplas de dados seguidas serão as linhas de 80 a 119, e usaremos duas linhas $''$ para representa-los, os estado em que o jogador está preso serão as linhas de 120 a 122, e usaremos um $p$ para indicar que está preso, e linhas $'$ para indicar a quanto tempo. Esta modelagem garante que teremos nas colunas de 40 a 119 as probabilidades do jogador tirar uma dupla de dados iguais, assim como de 120 a 122 as probabilidades dele ir preso e continuar preso.

Com esta modelagem também podemos encontrar facilmente a casa do jogador no tabuleiro usando módulo, pois se o jogador na casa 32 tirou uma dupla de dados iguais indo para o estado 72, sabemos que $32 = 72 mod(40)$ ou seja $n = n' mod(40) = n'' mod(40)$

Detalhes da modelagem:

* As linhas de 0 a 39 indicadas como $n$ representam as casas do tabuleiro
* As linhas de 40 a 79 indicadas como $n'$ representam os estado auxiliares, quando o jogador tira valores iguais nos dados
* As linhas de 80 a 119 indicadas como $n''$ representam os estado auxiliares, quando o jogador no estado $n'$ tira valores iguais nos dados
* As linhas de 120 a 122 representam o estado de quando o jogador vai para a prisão, assim podemos diferenciar a casa 20 (visita a prisão)
* A linha 120 será indicada como ($20p$)
* A linha 121 será indicada como ($20p'$) que representa o estado em que o jogador preso tira dados distintos e continua preso
* A linha 122 será indicada com ($20p''$) que representa o estado em que o jogador preso tira dados distintos e continua preso pela segunda vez
* As colunas de 0 a 122 seguem a mesma ideia das linhas, representando os possiveis estados futuros do jogador dependendo apenas de seu estado atual

Assim teremos a seguinte matriz de transição:

  $$ 
 P = \begin{bmatrix}
 P_{0,0} & P_{0,1}  & P_{0,2} & \cdots  & P_{0,122}  &  \\
 P_{1,0} & P_{1,1}  & P_{1,2} & \cdots  & P_{1,122}  &  \\
 P_{2,0} & P_{2,1}  & P_{2,2} & \cdots  & P_{2,122} \\
 \vdots  & \vdots  & \vdots   & \ddots  & \vdots  \\
 P_{122,0} & P_{122,1} & P_{122,2}  & \cdots  & P_{122,122}
\end{bmatrix}
$$

Onde cada $P_{ij}$ representa a probabilidade do jogador assumir o próximo estado.

Probabilidades dos dados:

Como temos dois dados honestos de 6 faces cada, isso significa que temos $6 x 6 = 36$ casos possiveis de resultados na rolagem dos dados. São eles: $(1,1)$, $(1,2)$, $(1,3)$, $(1,4)$,...,$(6,5)$,$(6,6)$.

Os valores da soma dos dados são representados no seguinte vetor:

$$\text{V = [2,3,4,5,6,7,8,9,10,11,12]}$$

Para encontrarmos a probabilidade de cada soma dos valores dos dados usamos:

$$\frac{Casos favoraveis}{Casos possiveis}$$

Logo nosso vetor de probabilidades dos dados é:

$$\text{V = [2,3,4,5,6,7,8,9,10,11,12]} → [\frac{1}{36},\frac{1}{18}, \frac{1}{12}, \frac{1}{9}, \frac{5}{36}, \frac{1}{6}, \frac{5}{36}, \frac{1}{9}, \frac{1}{12}, \frac{1}{18}, \frac{1}{36}]$$

A soma dos valores é a quantidade de casas no tabuleiro que o jogador anda, assim a probabilidade na matriz de transição da cadeia de Markov dele mudar de um estado $X_n$ para $X_{n+1}$ será as probabilidades acima distribuidas pelas coluna da matriz, por exemplo um jogador na casa 12 ($i = 12$), tem 0 probabilidade de ir para a casa 13 ($j=13$) ou qualquer casa além da 24 ($j=24$), já que a soma dos dados vai de 2 a 12, porém ele pode ir para a casa 14 ($j=14$) se tirar 1 + 1 nos dados, neste caso a probabilidade ($P_{12,14}$) será $\frac{1}{36}$.

Porém no caso do banco imobiliário quando o jogador tirar uma dupla de valores iguais nos dados, ele não irá se mover, terá que jogar novamente os dados, assim a probabilidade dele ir para a casa 14, estando na casa 12 é 0. Portanto se o jogador tirar qualquer dupla de valores iguais nos dados ele irá continuar na casa 12 tendo que jogar novamente os dados, em nossa modelagem o jogador ao tirar uma dupla irá para o estado "linha" ($'$) ou "duas linhas" ($''$) se for a segunda vez seguida, neste caso o jogador no estado 12 ao tirar 1 + 1 nos dados irá para o estado 52 ou $12'$. 

Como ele tem 6 casos favoraveis $(1,1),(2,2),(3,3),(4,4),(5,5),(6,6)$ de tirar uma duplas de valores iguais, ele tem probabilidade ($P_{12,52}$) $\frac{6}{36}$ ou $\frac{1}{6}$ de ir para o estado 52.

Assim em nossa modelagem, ao removermos as probabilidades de valores iguais dos dados e incluindo a probabilidade de obtermos 0 e 1, encontramos o seguinte vetor de valores e probabilidades:

$$\text{V = [0,1,2,3,4,5,6,7,8,9,10,11,12]} → [0,0,0,\frac{1}{18}, \frac{1}{18}, \frac{1}{9}, \frac{1}{9}, \frac{1}{6}, \frac{1}{9}, \frac{1}{9}, \frac{1}{18}, \frac{1}{18}, 0]$$

Portanto em cada linha de 0 ($i=0$) a 119 ($i=119$) de nossa matriz de transição o jogador terá, as probabilidades acima (distribuidas conforme a soma do estado atual modulo 40 com o index do vetor de probabilidades) de mudar para um dos estados das colunas de 0 ($j=0$) a 39 ($j=39$), e adicionando a probabilidade $\frac{1}{6}$ no estado "linha" ($'$) ou "duas linhas" ($''$) referente aquela linha, se linha 14 ($i=14$), estado 54 ($j=54$) ($14'$) tera probabilidade $P_{14,54}$ = $\frac{1}{6}$.

OBS: Caso o jogador possa dar a volta no tabuleiro no estado em que ele está atualmente a probabilidade que exceder o tabuleiro sera colocada no inicio da linha, exemplo, o jogar na casa 35 se tirar 4 + 5 nos dados terá completado uma volta no tabuleiro, e irá para a casa 4, assim a probabilidade $P_{35,4} = \frac{1}{9}$. Para popular o inicio da linha basta somar o indice do vetor de probabilidades com a linha atual modulo 40.

Nos casos das linhas de 80 ($i=80$) a 119 ($i=119$) colocamos a probabilidade $\frac{1}{6}$ no estado 120 ($j=120$), pois este representa a prisão, e um jogador no estado "duas linhas" ($''$) ao tirar uma dupla de valores iguais estará preso.

Nos casos especiais 120 ($i=120$) e 121 ($i=121$), o jogador terá probabilidade $\frac{5}{6}$ de rolar uma dupla de valores distintos nos dados, e continuar preso, estas probabilidades serão colocada nas colunas 121 ($j=121$) e 122 ($j=122$)respectivamente para cada estado, por fim terá probabilidade $\frac{1}{6}$ de voltar para a casa 20 ($j=20$) ao rolar uma dupla de valores iguais nos dados. Já para o caso 122 ($i=122$) coloa-se probabilidade 1 na coluna 20 ($j=10$), já que o jogador será obrigatoriamente solto neste estado.

Matriz de Transição:

$$
  M_{123x123} = \begin{bmatrix}
 0 & 0  & 0 & 1/18 & 1/18 &  \cdots & 1/6 &  \cdots & 0  &  \\
 0 & 0  & 0 & 0 & 1/18 &\cdots  & 0 &\cdots  & 0  &  \\
 0 & 0  & 0 & 0 & 0 & \cdots  & 0 &\cdots  & 0 \\
 \vdots  & \vdots  & \vdots   & \vdots  & \vdots  & \cdots  & \vdots & \ddots & \vdots  \\
 0 & 0 & 0 & 0 & 0  & \cdots  & 0 & 0 & 5/6 \\
 0 & 0 & 0 & 0 & 0  & \cdots  & 0 & 0 & 0
  \end{bmatrix}
$$ 

Abaixo segue o codigo para imprimir cada linha da matriz de transição:

In [4]:
#@title
#Funcao que gera a linha da matriz de transicao
@cache
def linhaMatrizTransicao(linha):

    linhaMarkov = np.zeros([123])

    #Vetor de probabilidades da soma dos valores diferentes dos dados
    #com inclusão dos estados 0 e 1
    probsCasas = [0, 0, 0, 1/18, 1/18, 1/9, 1/9, 1/6, 1/9, 1/9, 1/18, 1/18, 0]

    #Verificamos se a linha (estado) desejado é um caso de prisão ou não
    if (linha <= 119):

        #Variavel auxiliar para obter o valor do estado módulo 40
        linhaAux = linha % 40
        
        for i, p in enumerate(probsCasas):
            #Caso essa linha exceda as 40 casas aplica-se modulo 40 para voltar ao inicio da linha
            if (i + linhaAux > 39):
                linhaMarkov[(linhaAux+i)%40] = p
            #Caso contrario apenas associamos a coluna da linha a probabilidade
            else:
                linhaMarkov[linhaAux+i] = p

        #Verifica se o jogador está no estado normal ou no caso '
        if (linha < 80):
            #Associamos a probabilidade dele ir para o estado ' ou ''
            linhaMarkov[linha + 40] = 1/6

        #Verificamos se o jogador está no estado ''
        if (linha >= 80 and linha <= 119):
            #Associo a probabilidade dele ser preso (estado 120)
            linhaMarkov[120] = 1/6
    else:
        if (linha == 120):
            #Adicio a probabilidade dele ser solto
            linhaMarkov[20] = 1/6
            #Adicio a probabilidade dele continuar preso (estado 121)
            linhaMarkov[121] = 5/6

        #Verifico se o jogador está no segundo estado prisao
        #que se comporta de forma parecida com o anterior
        if (linha == 121):
            linhaMarkov[20] = 1/6
            linhaMarkov[122] = 5/6

        #Verificamos se o jogador está no terceiro estado prisao
        if (linha == 122):
            # Adicionamos probabilidade 1 do jogador ser solto e ir para o estado 20
            linhaMarkov[20] = 1
        
    return linhaMarkov

#Funcao que imprime linha de uma matriz com valores decimais em fracao, obrigado stack overflow
def imprimeLinhaMatriz(estado):
    linha = linhaMatrizTransicao(estado)

    linhaAux = []
    for j in range(len(linha)):
        linhaAux.append(str(Fraction(linha[j]).limit_denominator()))

    print('Linha do estado ' + str(estado))
    print(''.join(['{:6}'.format('' + str(item) + '  ') for item in range(0, 123)]))
    print(''.join(['{:6}'.format(item) for item in linhaAux]))

#Exemplos
print("\n")
imprimeLinhaMatriz(11)
print("\n")
imprimeLinhaMatriz(51)
print("\n")
imprimeLinhaMatriz(91)
print("\n")
imprimeLinhaMatriz(120)
print("\n")
imprimeLinhaMatriz(121)
print("\n")
imprimeLinhaMatriz(122)



Linha do estado 11
0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50    51    52    53    54    55    56    57    58    59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94    95    96    97    98    99    100   101   102   103   104   105   106   107   108   109   110   111   112   113   114   115   116   117   118   119   120   121   122   
0     0     0     0     0     0     0     0     0     0     0     0     0     0     1/18  1/18  1/9   1/9   1/6   1/9   1/9   1/18  1/18  0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     

Caso queira imprimir a matriz de transição inteira rode o código abaixo

In [5]:
#@title
def imprimeMatriz(m):
  auxM = []
  for i in range(len(m)):
    aux = []
    for j in range(len(m[i])):
      s = str(Fraction(m[i][j]).limit_denominator())
      aux.append(s)
    auxM.append(aux)
  
  print(''.join(['{:6}'.format('' + str(item) + '  ') for item in range(0, 123)]))
  print(('\n\n').join([''.join(['{:6}'.format(item) for item in row]) for row in auxM]))

matrizTransicao = np.zeros([123, 123])

for i in range(123):
    matrizTransicao[i] = linhaMatrizTransicao(i)

imprimeMatriz(matrizTransicao)

0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50    51    52    53    54    55    56    57    58    59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94    95    96    97    98    99    100   101   102   103   104   105   106   107   108   109   110   111   112   113   114   115   116   117   118   119   120   121   122   
0     0     0     1/18  1/18  1/9   1/9   1/6   1/9   1/9   1/18  1/18  0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     1/6   0     0     0  

# LETRA B

**LETRA B**
Enunciado:

b) Denote por (Xt)t∈Z ≥ 0 a posição do jogador no tabuleiro ao longo do tempo. Pelo item a), sabemos que essa sequência de variáveis aleatórias é descrita por uma cadeia de Markov. Construa um algoritmo para simular observações de tal cadeia, onde é possível controlar o número de iterações desejadas.

**LETRA B**
Resposta:

Para responder essa questão foi feito um algoritmo que usa a matriz de transição para obter o próximo estado do jogador.

Adotei, o estado inicial como 0, porém pode ser passado algum outro estado inicial desejado como um argumento opcional da função, com a matriz de transição sabemos exatamente quais são os possiveis estados futuros ($X_{n+1}$) de um jogador no estado atual $X_n$, assim se pegarmos a linha referente ao estado atual e eliminarmos os estado cuja probabilidade é 0, nos resto os estados possiveis para onde o jogador pode se mover.

Para isso criei um dicionario que associa o possivel estado futuro $j$ do jogador a sua probabilidade $P_{ij}$, após isso populo um vetor de tamanho 36, com esses estados $j$ de acordo com as suas probabilidades, por exemplo, o jogador no estado $i=0$ possui probabilidade $P_{0,3} = \frac{1}{18}$ logo coloco o estado $j=3$ duas vezes no vetor de tamanho 36. Após fazer para todos os $j's$ daquela linha sorteio um valor desse vetor, usando a função randint do python, este valor sorteado será o novo estado do jogador.

Faço isso $n$ vezes de acordo com a entrada do jogador, e ao final retorno a lista dos estados que o jogador percorreu nesta simulação, para melhor visualização, criei um vetor auxiliar especifico para exibição que é impresso por padrão, e nele caso o estado seja maior do que 39 aplico mod 40 para encontrar a casa em que o jogador estaria parado e adiciono um $'$ na string para melhor visualização, assim o estado $53$ é igual a $13'$ e o $93$ é igual a $13''$.

Para os estados de prisão coloco um $p$ e as "linhas" ($'$) para indicar que era prisão e os turnos.

Infelizmente usando a linha da matriz não posso indicar o resultado individual da rolagem dos dados, já que não a faço de fato, porém é possivel saber a soma dos dados ao fazer $X_n - X_{n+1}$.

OBS: Atribuí a função de simulação para uma variavel afim do COLAB não printar o return que será usado na letra C, o print do vetor de exibição está dentro da função simulação, por padrão é setado como True e pode ser desativado como uma variavel opcional, assim como o estado inicial do jogador.

In [6]:
#@title
#Esta função retorna o vetor dos estados possiveis que um jogador pode assumir
@cache
def probablidades(estado:int):

    #Retorna a linha da matriz de transicao daquele estado
    linhaMatriz = linhaMatrizTransicao(estado)

    estadosPossiveis = dict()

    #Armazena no dicionario os estado possiveis
    for i, p in enumerate(linhaMatriz):
        if (p != 0):
            estadosPossiveis[i] = p

    estadosPossiveisComPeso = []

    #Popula o vetor dos estados possiveis aplicando um peso de acordo com a 
    #probabilidade de cada estado
    for chave in estadosPossiveis:
        probabilidade = Fraction(estadosPossiveis.get(chave)).limit_denominator()
        k = probabilidade.numerator
        #Verifica se a fração possui 36 no denominador
        if (probabilidade.denominator != 36):
            #Caso não possua transforma o numerador, para antes da simplificação
            k = (36//probabilidade.denominator) * probabilidade.numerator
        #Adiciona no vetor o estado o numero de vezes igual ao numerador da probabilidade
        estadosPossiveisComPeso.extend([chave]*k)

    return estadosPossiveisComPeso

#Esta função apenas sorteia o proximo estado usando randint
def proximoEstado(estado:int):
    
    probs = probablidades(estado)

    novoEstado = probs[randint(0, 35)]

    return novoEstado

#Função que executa a simulação do jogo
def simulacao(t:int, imprimir=True, estadoInicial=0):
    jogadas = [estadoInicial]
    jogadasExibicao=[str(estadoInicial)]

    contador = 1
    while contador < t:
        #Determina o proximo estado do jogador
        jogada = proximoEstado(jogadas[contador-1])
        #Verifica se deve imprimir o vetor de exibicão
        if (imprimir == True):
          #Verifica se é um estado de prisao
          if (jogada > 119):
              jogadasExibicao.append("20p" + ((jogada % 40) * "'") )
          else:
              jogadasExibicao.append(str(jogada%40) + ((jogada // 40) * "'") )
        jogadas.append(jogada)
        contador += 1

    #Verifica se deve imprimir o vetor de exibicão
    if (imprimir == True):
        print(jogadasExibicao)

    return jogadas

s = simulacao(10000)

['0', '5', '12', '19', '22', '27', '36', '2', '12', '18', '26', "26'", "26''", '36', '2', '10', '17', "17'", '24', '30', '36', '39', '6', '12', '21', '32', '0', '8', '19', '24', '32', '1', "1'", "1''", '4', '7', "7'", '14', "14'", '24', '33', '0', '7', '16', '21', '31', '38', '6', '16', '22', '32', "32'", '39', '8', '18', '22', '26', '32', '1', '4', "4'", "4''", '14', '25', '34', '39', '8', '13', '22', "22'", "22''", '20p', "20p'", '20', '29', '37', '7', '12', '21', '28', "28'", '33', '1', '6', '12', '19', '30', '34', '3', '8', '11', '16', '26', '33', '0', '5', '12', '19', '27', '34', '4', '10', '20', '24', '30', '1', '6', '9', '16', "16'", '24', '31', "31'", "31''", '2', '6', '9', '18', '28', '37', '3', '8', '16', '25', '33', '38', '4', "4'", '12', '23', '28', "28'", '34', '2', "2'", '8', '15', '25', '30', "30'", '38', '5', '12', "12'", '19', '26', '33', '39', '3', '14', '19', '27', '34', '1', "1'", '8', "8'", '15', '22', '29', "29'", '38', '4', '13', "13'", '19', '22', '28', '37', "3

# LETRA C

**LETRA C**
Enunciado:

c) Seja Y a variável aleatória que representa a quantidade de passos que o jogador dá até a sua primeira prisão (tome cuidado, não é a primeira “visita à prisão”!). Obtenha uma aproximação para o valor esperado de Y, através de sucessivas simulações da cadeia usando o algoritmo construído no item b). Descreva detalhadamente o seu procedimento para realizar tal aproximação.

**LETRA C**
Resposta:

Para esta questão usarei o algoritmo construido em $b)$, que retorna as casas da simulação do jogo após $n$ jogadas.

Nesta questão fiquei em duvida se o desejado eram os passos até ele ser preso ou as jogadas (turnos), fiz considerando as jogadas (turnos).

OBS: Considerei os estados auxiliares ($'$) como parte de um turno, já que ele não se move quando tira as repetições dos dados e no banco imobiliario original ele só termina o turno após tirar valores diferentes no dados ou ir preso, caso tire as 3 repetições seguidas.

OBS 2: Chamarei turno de jogada.

Para encontrar o numero de jogadas antes da primeira prisão, percorremos o vetor da simulação e analisamos cada estado até a primeira prisão, se ele for menor que 40 somamos um no contador de jogadas, já que como descrito acima, nos interessa as jogadas em que ele se movimentou, e os estados auxiliares não se encaixam nessa situação, pois são insucessos da ida para prisão onde o jogador fica parado.

Ao chegarmos no estado 120 pela primeira vez, encontramos a primeira prisão assim podemos encerrar o laço e retornar o valor do contador. Caso não encontre nenhum 120, como em uma simulação de poucas jogadas onde não teve nenhuam prisão, retornamos 0.

Após isso populamos um vetor com os valores encontrados até a primeira prisão de cada simulação, fazemos isso $n$ vezes, e por fim fazemos a média aritmética dos valores desse vetor de acordo com a Lei dos Grandes Números ($LGN$), dessa forma obtemos o valor esperado de jogadas para a primeira prisão através do nosso algoritmo de simulação.

OBS 3: Usei o valor de 10 mil jogadas para cada simulação, garantindo assim que sempre ocorresse uma prisão em cada simulação.

In [7]:
#@title
#Funcao que conta as jogadas (turnos) de uma simulacao
#casos deseja contar os passos ate um estado especifico basta passa-lo como argumento
def contadorJogadas(simulacao, fim=123):

    contador = 0
    encontrouFim = False

    for j in simulacao:
        #Caso seja um estado de prisao, o laço e interrompido
        if (j == fim):
            encontrouFim = True
            break
        #Verifica se o estado corresponde a uma jogada, ou a um estado auxiliar
        elif (j < 40):
            contador += 1

    return contador if encontrouFim else 0



#Funcao que calcula o valor esperado de Y
def valorEsperadoY(t, imprimir=True):
  y = []

  contador = 0
  while contador < t:
      s = simulacao(10000, False)
      #Conta as jogadas ate o estado da prisao (120)
      p = contadorJogadas(s, 120)

      y.append(p)

      contador += 1

  #Imprime o vetor com os numeros de passos caso desejado
  if (imprimir):
    print("Y=y = " + str(y))

  #Retorna a media aritmetica
  return sum(y)/len(y)

print("JOGADAS (TURNOS) ANTES DA PRIMEIRA PRISAO:\n")

e = valorEsperadoY(3000)

print("\nValor esperado de Y = " + str(round(e)))

JOGADAS (TURNOS) ANTES DA PRIMEIRA PRISAO:

Y=y = [15, 30, 48, 75, 145, 59, 998, 231, 233, 9, 689, 415, 14, 311, 53, 200, 34, 263, 134, 184, 58, 122, 49, 389, 121, 113, 142, 9, 73, 115, 242, 111, 215, 138, 265, 131, 442, 4, 49, 11, 16, 177, 222, 634, 25, 123, 353, 59, 147, 277, 89, 49, 321, 32, 115, 340, 104, 352, 214, 217, 113, 5, 241, 125, 300, 140, 31, 990, 799, 245, 99, 310, 55, 12, 93, 101, 158, 40, 72, 683, 582, 925, 215, 64, 54, 22, 31, 336, 141, 49, 17, 105, 37, 285, 262, 507, 70, 467, 78, 282, 171, 69, 49, 395, 152, 129, 169, 328, 84, 7, 94, 138, 90, 530, 666, 112, 148, 198, 117, 109, 163, 67, 263, 138, 117, 452, 183, 345, 87, 88, 648, 179, 79, 743, 45, 38, 87, 165, 5, 474, 105, 347, 2, 1023, 278, 192, 47, 35, 152, 49, 947, 29, 195, 253, 217, 86, 450, 465, 104, 152, 52, 143, 686, 351, 441, 159, 32, 142, 95, 355, 242, 161, 33, 508, 148, 512, 32, 728, 115, 162, 26, 271, 285, 354, 145, 19, 63, 9, 42, 313, 335, 251, 120, 363, 90, 165, 17, 193, 7, 21, 67, 322, 82, 76, 240, 30, 223,

# LETRA D

**LETRA D**
Enunciado:

d) Justifique matematicamente porque o procedimento realizado no item c) funciona para aproximar o valor esperado de Y.

**LETRA D**
Resposta:

O procedimento acima funciona para encontrar o valor esperado de Y, porque de acordo com a Lei dos Grandes Números ($LGN$) a média aritmética dos resultados de uma experiência tende a se aproximar do valor esperado conforme mais tentivas são realizadas. 

No caso das jogadas até a primeira prisão, Y é uma variavel aleatoria independente, assim podemos modelar o problema como uma ditribuição geométrica para encontra o valor esperado e comparar com o valor encontrado na letra $c)$.

Em uma distribuição geométrica são necessárias $n$ tentativas antes do primeiro sucesso.

Neste caso o primeiro sucesso seria a primeira ida a prisão, desconsiderando os estado auxiliares que são casos de insucesso da ida a prisão e considerando somente os estados de 0 a 39 que são as jogadas onde o jogadas se movimentou, temos que a probabilidade do jogador ir para a prisao é de $\frac{1}{216}$, pois a chance de tirar uma dupla de valores iguais nos dados é $\frac{1}{6}$ sendo assim isso precisa ocorrer $3$ vezes logos:

$$\frac{1}{6}*\frac{1}{6}*\frac{1}{6}=\frac{1}{216}$$

Sabendo que se trata de uma distribuição geométrica, podemos usar a fórmula de esperança:

$$E(Y)=\frac{1}{P}=\frac{1}{\frac{1}{216}}=216$$

O que nos mostra que o procedimento realizado no item $c)$ calculado a partir da média aritmética está de fato próximo do valor esperado ao calcularmos pela distribuição geométrica.

# LETRA E

**LETRA E**
Enunciado:

e) Seja Z a variável aleatória que representa a posição do jogador no tabuleiro após transcorrido um tempo suficientemente longo, ou seja, Xt quando t → ∞. Encontre a função massa de probabilidade de Z. Em média, em qual casa o jogador passa mais tempo? Discuta os resultados obtidos.

**LETRA E**
Resposta:

Para encontrarmos a F.M.P. podemos usar a propriedade da cadeia de Markov de que a matriz de transição pode ser escrita como um vetor de linha estocástico $x$ com $x_{n + 1} = x_nP$. 

Assim, temos que para um instante de tempo $n+2$ podemos escrever:

$$x_{n+2} = (x_{n+1})P = (x_nP)P = (x_n)P^2  $$ 

Logo:

$$x_{n+k} = (x_{n+(k-1)})P = (x_nP)P^{k-1} = (x_n)P^k$$ 

Nosso vetor inicial por sua vez é $[1, 0, 0, 0, ..., 0]$ de tamanho 123. 

Realizando $(x_n)P^k$ com $n → ∞$, obtemos a nossa F.M.P. para Z.

Após isso para obter o valor esperado aplicamos a fórmula de esperança:

$$\sum_{z=0}^{39} = z_{i}p_{z_i}$$

No código abaixo utilizei o valor de 1 milhão para $t$.

Primeiro chamamos uma função que constrói a matriz de transição, depois uma função que multiplica a matriz de transição pelo vetor inicial $t$ vezes, e após isso como ela era originalmente uma matriz 123x123, soma as probabilidades dos estados auxiliares 40 a 122 ao estados padrões 0 a 39, pois os estados auxiliares ainda são as mesmas casas Z, ao ir do estado 12 para o 52 ($12'$) o jogador não se moveu continou na mesma casa. Assim somamos usando o módulo as probabilidades dos estados auxiliares aos estados $n$ e ao final somamos as probabalidades do jogador estar preso, estados 120, 121 e 122, ao estado 20, que representa a casa da prisão.

In [8]:
#@title
#Funcao que constroi a matriz de transicao usando a funcao de linha da matriz
#usada na questao a)
def matrizTransicao():

    matriz = np.zeros([123, 123])
    for i in range(0, 123):
        linha = linhaMatrizTransicao(i)
        matriz[i] = linha

    return matriz

#Funcao que calcula a FMP
def probabilidadesAposTempo(t:int):
    m = matrizTransicao()

    posicao = np.zeros([123])
    posicao[0] = 1

    #Executa a multiplicacao do vetor inicial pela matriz de transicao t vezes
    contador = 0
    while contador < t:
        posicao = posicao.dot(m)
        contador += 1

    probabilidades = dict()

    #Percorre o vetor apos a multiplicacao da matriz de transicao somando as
    #probabilidades dos estados auxiliares as casas de 0 a 39
    for i, z in enumerate(posicao):

        if (i >= 40 and i < 120):
            probabilidades["Z(" + str(i%40) + ")"] += z
        elif (i >= 120):
            probabilidades["Z(" + str(20) + ")"] += z
        else:
            probabilidades["Z(" + str(i) + ")"] = z

    return probabilidades

#Funcao que calcula a esperanca
def esperancaZ(probablidades):

    e = 0
    #Percorre o dicionario das probabilidades de Z para realizar o
    #somatorio da formulade esperanca
    for i, p in enumerate(probabilidades.values()):
        e += (i*p)

    return round(e)

probabilidades = probabilidadesAposTempo(1000000)

print("F.M.P. de Z = " + str(probabilidades))

print("\n")

e = esperancaZ(probabilidades)

print("E(Z) = " + str(e))

F.M.P. de Z = {'Z(0)': 0.02467988208940804, 'Z(1)': 0.024664198447949863, 'Z(2)': 0.024660284996556443, 'Z(3)': 0.024648396083852903, 'Z(4)': 0.024634861825702768, 'Z(5)': 0.02462078840022994, 'Z(6)': 0.02459653201647888, 'Z(7)': 0.024578667047704433, 'Z(8)': 0.02455908226152839, 'Z(9)': 0.024544639036735157, 'Z(10)': 0.02452923177704426, 'Z(11)': 0.024515203684677803, 'Z(12)': 0.024499879332269006, 'Z(13)': 0.024483386916049576, 'Z(14)': 0.024466587116676604, 'Z(15)': 0.02444935671402126, 'Z(16)': 0.024432949926909156, 'Z(17)': 0.02441646684184245, 'Z(18)': 0.02440074670938408, 'Z(19)': 0.024384880944090415, 'Z(20)': 0.03865625995633891, 'Z(21)': 0.024352901101980698, 'Z(22)': 0.02433664380225919, 'Z(23)': 0.02462460327958881, 'Z(24)': 0.02460833444937048, 'Z(25)': 0.024896455224357444, 'Z(26)': 0.02490057616205139, 'Z(27)': 0.025209008711133542, 'Z(28)': 0.024949318567940745, 'Z(29)': 0.024995192237088893, 'Z(30)': 0.024798702387960187, 'Z(31)': 0.024871420421462058, 'Z(32)': 0.02468

# REFERÊNCIAS

[Como fazer cache em Python](https://wiki.python.org.br/CacheDeFuncoes)

[Dominating Monopoly Using Markov Chains](https://www.youtube.com/watch?v=Mh5r0a23TO4)

[Matriz de Probabilidades de Transição - Cadeia de Markov](https://www.youtube.com/watch?v=NKESrlCLBF0)

[Exploring strategies in Monopoly using Markov chains and simulation
by Albert Nilsson](http://uu.diva-portal.org/smash/get/diva2:1471765/FULLTEXT01.pdf)

[Lei dos grandes números](https://pt.wikipedia.org/wiki/Lei_dos_grandes_números)

[Cadeias de Markov](https://pt.wikipedia.org/wiki/Cadeias_de_Markov)

[stackoverflow](https://stackoverflow.com)
